Proyecto prueba TW LATAM

Para que este proyecto funcione

In [1]:
# Habilita la recarga automática de módulos en Jupyter Notebook.
# Esto mejora el flujo de trabajo de desarrollo al aplicar automáticamente los cambios realizados
# en los módulos importados sin necesidad de reiniciar manualmente el kernel.
%reload_ext autoreload

# Configura el modo de recarga automática.
# La opción "2" asegura que todos los módulos se recarguen automáticamente después de cualquier cambio.
%autoreload 2

# Mensaje de confirmación para indicar que la funcionalidad de autoreload está activa.
print("IPython autoreload activated")

IPython autoreload activated


Mount a content/drive de la cuenta de Google Drive donde almacenaremos el proyecto y la data de Tweets que trabajaremos

In [2]:
# Montar Google Drive
# En este caso no instalamos ninguna librería para Drive puesto estamos utilizando Google Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Cambia el directorio de trabajo a la carpeta especificada en Google Drive.
Esto es útil cuando se trabaja en Google Colab para organizar proyectos y facilitar el acceso a los archivos.

In [3]:
cd /content/drive/MyDrive/tw-latam-test-b/tw-latam-test-project

/content/drive/MyDrive/tw-latam-test-b/tw-latam-test-project


In [4]:
# Librerías necesarias
!pip install -r requirements.txt

In [5]:
import gdown
import zipfile

# Define la ruta
file_directory = '/content/drive/MyDrive/tw-latam-test-b/tw-latam-test-project/tweets_extracted/'

# Define nombre de archivo
file_name = 'farmers-protest-tweets-2021-2-4.json'

# concatenamos la ruta y el nombre del archivo
file_path = file_directory + file_name
print(file_path)

# ID del archivo en Drive para descargar Tweets en formato ZIP
file_id = "1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis"

# URL generica para descargas Drive
download_url = f"https://drive.google.com/uc?id={file_id}"

# Nombre para el archivo descargado
zip_output = "tweets.json.zip"

# Descargar el archivo
gdown.download(download_url, zip_output, quiet=False)

# Descomprimir el archivo
with zipfile.ZipFile(zip_output, 'r') as zip_ref:
    zip_ref.extractall("tweets_extracted")

print("Archivo descomprimido en la carpeta 'tweets_extracted'")

/content/drive/MyDrive/tw-latam-test-b/tw-latam-test-project/tweets_extracted/farmers-protest-tweets-2021-2-4.json


Downloading...
From (original): https://drive.google.com/uc?id=1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis
From (redirected): https://drive.google.com/uc?id=1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis&confirm=t&uuid=d03d0c6b-51d3-4a2a-a22c-50486789df53
To: /content/drive/MyDrive/tw-latam-test-b/tw-latam-test-project/tweets.json.zip
100%|██████████| 60.4M/60.4M [00:01<00:00, 30.6MB/s]


Archivo descomprimido en la carpeta 'tweets_extracted'


Se importan los modulos python junto con sus funciones  

In [6]:
%run src/q1_memory.py
%run src/q1_time.py
%run src/q2_memory.py
%run src/q2_time.py
%run src/q3_memory.py
%run src/q3_time.py
%run src/utilidades.py

Define las funciones time call y memory_call para modularizar las llamadas de las funciones y las respuesta, evitando la repetición de codigo

In [7]:
# line_profiler y memory_profiler
import line_profiler
from memory_profiler import profile
from memory_profiler import memory_usage

# definición de la función time_call para el registro de tiempos
@time_it
def time_call(function, file_path):
    """
    Perfila el tiempo de ejecución de la función especificada usando line_profiler y devuelve su resultado.

    Args:
        function (callable): Función a perfilar.
        file_path (str): Ruta del archivo que procesará la función.

    Returns:
        Any: Resultado de la función perfilada.
    """
    from line_profiler import LineProfiler

    # Crear una nueva instancia del perfilador para evitar unificar todo el log de hits
    profiler = LineProfiler()

    # Agregar explícitamente la función al perfilador
    profiler.add_function(function)

    # Usar un diccionario para capturar las variables locales
    local_vars = {"file_path": file_path, "result": None, "function": function}

    # Ejecutar y perfilar la función
    profiler.runctx("result = function(file_path)", globals(), local_vars)

    # Mostrar estadísticas del perfilador
    print(f"=== Estadísticas de Line Profiler para {function.__name__} ===")
    profiler.print_stats()
    print("=== Fin de las Estadísticas ===")

    # Retornar el resultado de la función
    return local_vars["result"]


# definición de la función memory_call para el registro de memoria
@time_it
def memory_call(function, file_path):
    """
    Perfila el uso de memoria de la función especificada y devuelve su resultado.

    Args:
        function_name (callable): Función a perfilar.
        file_path (str): Ruta del archivo que procesará la función.

    Returns:
        Any: Resultado de la función perfilada.
    """
    def wrapper():
        return function(file_path)

    # Perfilar memoria usando memory_usage
    mem_usage, result = memory_usage(wrapper, retval=True, interval=0.1)

    # Mostrar estadísticas de memoria
    print("=== Estadísticas de Memory Profiler ===")
    print(f"Uso máximo de memoria: {max(mem_usage):.2f} MiB")
    print(f"Uso inicial de memoria: {mem_usage[0]:.2f} MiB")
    print(f"Uso final de memoria: {mem_usage[-1]:.2f} MiB")
    print("=== Fin de las Estadísticas ===")

    return result

Las top 10 fechas donde hay más tweets más el usuario (username) con más tweets y su cantidad

In [8]:
# Llamada a 10 fechas con mayor cantidad de tweets y el usuario traves de la funcion time_call con la estadistica de tiempo de ejecucion
top_tweets_time = time_call(q1_time, file_path)
print(f"Resultado final: {top_tweets_time}")

=== Estadísticas de Line Profiler para q1_time ===
Timer unit: 1e-09 s

Total time: 22.8506 s
File: /content/drive/MyDrive/tw-latam-test-b/tw-latam-test-project/src/q1_time.py
Function: q1_time at line 7

Line #      Hits         Time  Per Hit   % Time  Line Contents
     7                                           def q1_time(file_path: str) -> List[Tuple[datetime.date, str, int]]:
     8                                               # Diccionario para almacenar el conteo de tweets por fecha y usuario
     9         1       4995.0   4995.0      0.0      tweet_count_by_date_user = defaultdict(lambda: defaultdict(int))
    10         1        489.0    489.0      0.0      chunksize = 10000  # Leer 10,000 líneas a la vez
    11                                           
    12         1        766.0    766.0      0.0      def process_chunk(chunk):
    13                                                   # Convertir la columna 'date' a datetime y extraer 'username'
    14                  

Los top 10 emojis más usados con su respectivo conteo

In [9]:
# Llama a la funcion q2_time a traves de time_call para obtener la estadistica de tiempo en el proceso de busqueda de los top 10 emojis
top_emojis_time_valor = time_call(q2_time, file_path)
print(f"Resultado final: {top_emojis_time_valor}")

=== Estadísticas de Line Profiler para q2_time ===
Timer unit: 1e-09 s

Total time: 20.5571 s
File: /content/drive/MyDrive/tw-latam-test-b/tw-latam-test-project/src/q2_time.py
Function: q2_time at line 12

Line #      Hits         Time  Per Hit   % Time  Line Contents
    12                                           def q2_time(file_path: str) -> List[Tuple[str, int]]:
    13         1      15682.0  15682.0      0.0      emoji_counter = Counter()
    14                                           
    15         2   55861398.0    3e+07      0.3      with open(file_path, 'r', encoding='utf-8') as file:
    16    117408 1680537272.0  14313.7      8.2          for line in file:
    17    117407   30644747.0    261.0      0.1              try:
    18    117407 5555520424.0  47318.5     27.0                  tweet = json.loads(line)  # Cargar cada línea como un objeto JSON
    19    117407   77575085.0    660.7      0.4                  content = tweet.get('content', '')  # Obtener el conteni

El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos

In [10]:
# Llama a la funcion q3_time a traves de time_call para obtener la estadistica de tiempo
# Proceso de busqueda de los usuarios mas influyentes
top_mentions_time = time_call(q3_time, file_path)
print(f"Resultado final: {top_mentions_time}")

=== Estadísticas de Line Profiler para q3_time ===
Timer unit: 1e-09 s

Total time: 6.6155 s
File: /content/drive/MyDrive/tw-latam-test-b/tw-latam-test-project/src/q3_time.py
Function: q3_time at line 13

Line #      Hits         Time  Per Hit   % Time  Line Contents
    13                                           def q3_time(file_path: str) -> List[Tuple[str, int]]:
    14                                               # inicia un collection de contador para menciones
    15         1      11132.0  11132.0      0.0      mention_counter = Counter()
    16                                           
    17                                               # carga tweets del archivo JSON línea por línea
    18         2   55487657.0    3e+07      0.8      with open(file_path, 'r', encoding='utf-8') as file:
    19    117408 1171925845.0   9981.7     17.7          for line in file:
    20    117407   27447464.0    233.8      0.4              try:
    21    117407 4531375081.0  38595.4     68.5

Las top 10 fechas donde hay más tweets más el usuario (username) con más tweets y su cantidad

In [ ]:
# Llamada a 10 fechas con mayor cantidad de tweets y el usuario traves de la funcion memory_call con la estadistica de memoria utilizada
top_tweets_memory = memory_call(q1_memory, file_path)
print(f"Resultado final: {top_tweets_memory}")

Los top 10 emojis más usados con su respectivo conteo

In [ ]:
# Llama a la funcion q2_memory a traves de la función memory cal para obtener la estadistica de memoria en el proceso busqueda top emojis
top_emoji_memory = memory_call(q2_memory, file_path)
print(f"Resultado final: {top_emoji_memory}")

El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos

In [ ]:
# Llama a la funcion q3_memory a traves de la función memory cal para obtener la estadistica de memoria en el proceso busqueda top mentions
top_mentions_time = memory_call(q3_memory, file_path)
print(f"Resultado final: {top_mentions_time}")